In [4]:
pip install helium

^C
Note: you may need to restart the kernel to use updated packages.


  Using cached helium-5.1.1.tar.gz (40 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached selenium-4.33.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached trio-0.30.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached certifi-2025.4.26-py3-none-any.whl.metadata (2.5 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Usin


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

def post_to_facebook(fb_email: str,
                     fb_password: str,
                     image_path: str,
                     description: str,
                     headless: bool = False,
                     disable_notifications: bool = True):
    """
    Automatisation Selenium pour publier une image + description sur Facebook.
    
    :param fb_email:       Adresse e-mail ou numéro de téléphone Facebook.
    :param fb_password:    Mot de passe Facebook.
    :param image_path:     Chemin absolu vers le fichier image à poster.
    :param description:    Texte à publier.
    :param headless:       Si True, lance Chrome en mode headless.
    :param disable_notifications: Si True, bloque les popups notifications.
    """

    # === Configuration des options Chrome ===
    options = Options()
    if headless:
        options.add_argument("--headless")
    if disable_notifications:
        options.add_argument("--disable-notifications")

    # === Lancement du navigateur ===
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    wait = WebDriverWait(driver, 15)

    try:
        # === Accès et connexion ===
        driver.get("https://www.facebook.com")
        wait.until(EC.presence_of_element_located((By.ID, "email")))\
            .send_keys(fb_email)
        driver.find_element(By.ID, "pass").send_keys(fb_password)
        driver.find_element(By.NAME, "login").click()

        # === Attente de la page d’accueil ===
        wait.until(EC.url_contains("facebook.com"))
        time.sleep(3)

        # === Ouvrir le formulaire de publication ===
        post_button = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//span[contains(text(),'Quoi de neuf')]/ancestor::div[@role='button']")))
        post_button.click()

        # === Renseigner la description ===
        text_area = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//div[@role='textbox']")))
        text_area.send_keys(description)

        # === Cliquer sur "Photo/Vidéo" ===
        photo_button = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//div[@aria-label='Photo/Vidéo']")))
        photo_button.click()

        # === Uploader l’image ===
        file_input = wait.until(EC.presence_of_element_located(
            (By.XPATH, "//input[@type='file']")))
        file_input.send_keys(os.path.abspath(image_path))
        time.sleep(3)

        # === Publier ===
        publish_btn = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//div[@aria-label='Publier']")))
        publish_btn.click()

        # === Pause finale ===
        time.sleep(5)
        print("✅ Publication Facebook terminée avec succès !")

    except Exception as e:
        print(f"🚫 Échec de la publication Facebook : {e}")

    finally:
        driver.quit()


# === Exemple d’utilisation ===
if __name__ == "__main__":
    FB_EMAIL    = "ektkawtar@gmail.com"
    FB_PASSWORD = "123hajar"
    IMAGE_PATH  = "tapis.jpeg"
    DESCRIPTION = "Tapis fait main en laine pure. Disponible maintenant !"

    post_to_facebook(FB_EMAIL, FB_PASSWORD, IMAGE_PATH, DESCRIPTION, headless=False)


✅ Publication Facebook terminée.
